In [1]:
import torch
import torch.nn as nn
import numpy as np

## PyTorch `torch.nn` 模块分类及常用子模块一览

| 模块分类       | 子模块（示例）                                                                 | 功能说明 |
|----------------|----------------------------------------------------------------------------------|----------|
| **线性层**     | `nn.Linear`, `nn.Bilinear`, `nn.LazyLinear`                                     | 实现线性变换（仿射变换），常用于全连接层 |
| **卷积层**     | `nn.Conv1d`, `nn.Conv2d`, `nn.Conv3d`, `nn.ConvTranspose2d`, `nn.LazyConv2d`     | 用于提取局部特征，适用于图像、时序、体数据等 |
| **池化层**     | `nn.MaxPool1d`, `nn.MaxPool2d`, `nn.AvgPool2d`, `nn.AdaptiveAvgPool2d`           | 对特征图进行下采样，减小空间维度，提取主特征 |
| **归一化层**   | `nn.BatchNorm1d`, `nn.LayerNorm`, `nn.GroupNorm`, `nn.InstanceNorm2d`            | 缓解内部协变量偏移，加快训练收敛速度 |
| **激活函数**   | `nn.ReLU`, `nn.LeakyReLU`, `nn.Sigmoid`, `nn.Tanh`, `nn.Softmax`, `nn.GELU`      | 引入非线性，增强模型表达能力 |
| **循环层**     | `nn.RNN`, `nn.LSTM`, `nn.GRU`, `nn.LSTMCell`, `nn.GRUCell`                       | 用于处理时序数据和序列建模任务 |
| **注意力模块** | `nn.MultiheadAttention`                                                         | 实现多头自注意力机制，广泛用于Transformer结构 |
| **损失函数**   | `nn.CrossEntropyLoss`, `nn.MSELoss`, `nn.L1Loss`, `nn.BCELoss`, `nn.NLLLoss`     | 衡量预测值与真实值的差异，指导模型优化 |
| **容器模块**   | `nn.Sequential`, `nn.ModuleList`, `nn.ModuleDict`, `nn.ParameterList`            | 用于组织和管理多个子模块（模型结构） |
| **正则化层**   | `nn.Dropout`, `nn.AlphaDropout`, `nn.Dropout2d`                                  | 随机丢弃神经元，防止过拟合 |
| **嵌入层**     | `nn.Embedding`, `nn.EmbeddingBag`                                                | 用于将离散变量（如单词索引）映射为连续向量 |
| **Transformer**| `nn.Transformer`, `nn.TransformerEncoder`, `nn.TransformerDecoder`                | Transformer 编码器/解码器模块 |
| **初始化方法** | `nn.init.xavier_uniform_`, `nn.init.kaiming_normal_`, `nn.init.constant_`        | 用于权重初始化，提高训练稳定性 |
| **并行模块**   | `nn.DataParallel`, `nn.parallel.DistributedDataParallel`                         | 支持多 GPU 并行训练 |
| **函数式接口** | `torch.nn.functional`                                                            | 提供激活函数、卷积、池化、归一化等操作的函数式版本 |
| **工具类**     | `nn.Parameter`, `nn.utils.clip_grad_norm_`, `nn.utils.rnn.pack_padded_sequence`  | 参数封装、梯度裁剪、变长序列处理等实用工具 |
| **其他模块**   | `nn.Identity`, `nn.Flatten`, `nn.Unflatten`, `nn.Softmin`, `nn.Softsign`          | 用于辅助结构调整或特殊功能 |

# Dropout 使用方式对比（模块化 vs 函数式）

| 使用方式                    | 是否自动根据 `model.train()/eval()` 切换 | 是否具有状态 | 说明                                                               | 推荐场景         |
|-----------------------------|--------------------------------------------|---------------|--------------------------------------------------------------------|------------------|
| `nn.Dropout`                | ✅ 是                                       | ✅ 有 `training` 状态 | 作为 Module 的一部分，自动切换训练和测试阶段行为                    | ✅ 推荐用于模型类 |
| `torch.nn.functional.dropout` | ❌ 否，需手动设置 `training=True/False`     | ❌ 无状态      | 函数式接口，无状态追踪，灵活但容易在测试时出错                     | 可用于简单函数   |

## 示例代码

### ✅ 推荐：使用 `nn.Dropout` 模块
```python
import torch.nn as nn

class MyModel(nn.Module):
	def __init__(self):
		super().__init__()
		self.dropout = nn.Dropout(p=0.5)

	def forward(self, x):
		return self.dropout(x)

model = MyModel()
model.train()  # 启用 Dropout
output_train = model(torch.ones(1, 10))

model.eval()   # 关闭 Dropout（推理阶段）
output_eval = model(torch.ones(1, 10))
```

### ⚠ 注意：使用 `F.dropout` 函数
```python
import torch.nn.functional as F

def forward(x, training=True):
	return F.dropout(x, p=0.5, training=training)

output_train = forward(torch.ones(1, 10), training=True)
output_eval = forward(torch.ones(1, 10), training=False)
```

## 总结
> 若使用 `F.dropout`，务必在测试阶段传入 `training=False`，否则 Dropout 仍然激活，会严重影响模型推理性能。